In [1]:
import reservoirflow as rf
import numpy as np

In [2]:
dir(rf.scalers)

['Dummy',
 'MinMax',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_dummy',
 '_minmax',
 '_scaler']

In [3]:
dir(rf.models)

['BlackOil',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_black_oil',
 '_model']

In [4]:
rf.__version__

'0.0.1'

In [5]:
dir(rf)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_base',
 'factors_dict',
 'fluids',
 'grids',
 'models',
 'scalers',
 'units_dict',
 'utils',
 'wells']

In [6]:
dir(rf)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_base',
 'factors_dict',
 'fluids',
 'grids',
 'models',
 'scalers',
 'units_dict',
 'utils',
 'wells']

In [7]:
rf.units_dict

{'field': {'transmissibility': 'STB/D-psi',
  'error': 'STB/D',
  'pressure': 'Psia',
  'time': 'Days',
  'rate': 'STB/Day'},
 'metric': {'transmissibility': 'M3/D-bar', 'error': 'M3/D'}}

In [8]:
rf.fluids.SinglePhase()

{'unit': 'field', 'units': {'transmissibility': 'STB/D-psi', 'error': 'STB/D', 'pressure': 'Psia', 'time': 'Days', 'rate': 'STB/Day'}, 'factors': {'transmissibility conversion': 0.001127, 'gravity conversion': 0.00021584000000000002, 'gravitational acceleration': 32.174, 'volume conversion': 5.614583}, 'dtype': 'double', 'verbose': False, 'rho': 0, 'g': 0.0, 'comp': 0, 'comp_type': 'incompressible'}

In [9]:
type(rf.scalers.MinMax((1, 1)))

reservoirflow.scalers._minmax.MinMax

In [10]:
def create_model_example_7_1():
    grid = rf.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = rf.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = rf.models.BlackOil(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model


def create_model():
    grid = rf.grids.Cartesian(
        nx=3,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=20,
        phi=0.27,
        kx=1,
        ky=1,
        kz=0.1,
        comp=1 * 10**-6,
        dtype="double",
    )
    fluid = rf.fluids.SinglePhase(
        mu=0.5, B=1, rho=50, comp=1 * 10**-5, dtype="double"
    )
    model = rf.models.BlackOil(
        grid, fluid, pi=6000, dt=5, start_date="10.10.2018", dtype="double"
    )

    n_samples = 1
    cells_id = model.grid.get_cells_id(False, True)[-1].flatten()
    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=-300, pwf=100, s=1.5, r=3.5)

    wells = np.random.choice(cells_id, n_samples, False)
    for id in wells:
        model.set_well(id=id, q=100, s=0, r=3.5)

    return model


model = create_model()
model.run(10, isolver="cgs")

[info] Simulation run started: 10 timesteps.


[step]: 10 - : 100%|██████████| 10/10 [00:00<00:00, 55.30steps/s]

[info] Simulation run of 10 steps finished in 0.19 seconds.
[info] Material Balance Error: 9.880984919163893e-15.


In [11]:
model.units

{'transmissibility': 'STB/D-psi',
 'error': 'STB/D',
 'pressure': 'Psia',
 'time': 'Days',
 'rate': 'STB/Day'}

In [12]:
arr = model.grid.get_cells_center(True, False)
print(arr.shape)
print(arr.min(axis=0))
arr

(5, 3)
[150. 175.  10.]


array([[ 150.,  175.,   10.],
       [ 450.,  175.,   10.],
       [ 750.,  175.,   10.],
       [1050.,  175.,   10.],
       [1350.,  175.,   10.]])

In [13]:
np.count_nonzero(np.array([1, 2, 0, 0]) == 0)

2

In [14]:
scaler = rf.scalers.MinMax((-1, 1)).fit(arr, axis=0)
scaler.transform(arr)

C:\Users\abugrzka\Projects\reservoirflow\reservoirflow\scalers\_minmax.py:38: RuntimeWarning: invalid value encountered in true_divide
  self.vmax - self.vmin


array([[-1. ,  nan,  nan],
       [-0.5,  nan,  nan],
       [ 0. ,  nan,  nan],
       [ 0.5,  nan,  nan],
       [ 1. ,  nan,  nan]])

In [15]:
t = np.arange(0, (model.tstep + 1) * model.dt, model.dt)
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [16]:
t

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

In [17]:
t, x = model.get_domain(False, True)
t, x

(array([[ 0],
        [ 5],
        [10],
        [15],
        [20],
        [25],
        [30],
        [35],
        [40],
        [45],
        [50]]),
 array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]))

In [18]:
rf.scalers.MinMax((-1, 1)).fit_transform(x, axis=0)

array([[-1. ],
       [-0.5],
       [ 0. ],
       [ 0.5],
       [ 1. ]])

In [19]:
rf.scalers.MinMax((0, 1)).fit_transform(t, axis=0)

array([[0. ],
       [0.1],
       [0.2],
       [0.3],
       [0.4],
       [0.5],
       [0.6],
       [0.7],
       [0.8],
       [0.9],
       [1. ]])

In [20]:
model.get_centers(False)

(array([[ 150.],
        [ 450.],
        [ 750.],
        [1050.],
        [1350.]]),
 ['x'])

In [21]:
config = {"time": ["MinMax", (0, 1)], "space": ["minMax"]}

In [22]:
model.set_scalers({"time": ["minmax", (-10, 10)], "rate": ["minmax", None]})
model.scalers_dict

{'time': ['MinMax', (-10, 10)],
 'space': ['MinMax', (-1, 1)],
 'pressure': ['MinMax', (-1, 1)],
 'rate': [None, None]}

In [23]:
model.update_scalers(False)
model.pressures_scaler.transform(model.pressures[1])
# model.space_scaler.transform(model.get_centers(False)[0])

array([0.66761452, 0.86235936, 0.15077701, 0.56868551, 0.66761452])

In [24]:
model.get_df(
    scale=True,
    melt=False,
    boundary=True,
    units=True,
    columns=["time", "cells", "wells"],
)

,Time [Scaled],Q1 [Scaled],Q2 [Scaled],P0 [Scaled],P1 [Scaled],P2 [Scaled],P3 [Scaled],P4 [Scaled],Qw2 [Scaled],Qw1 [Scaled],Pwf2 [Scaled],Pwf1 [Scaled]
Step,,,,,,,,,,,,
0,-10.0,0.0,0.000000,0.667615,0.667615,0.667615,0.667615,0.667615,0.000000,0.0,0.667615,0.667615
1,-8.0,100.0,-215.922133,0.667615,0.862359,0.150777,0.568686,0.667615,-215.922133,100.0,-4.093168,2.400537
2,-6.0,100.0,-199.743106,0.667615,0.958959,-0.167221,0.427824,0.667615,-199.743106,100.0,-4.093168,2.497136
3,-4.0,100.0,-189.147301,0.667615,0.997204,-0.375482,0.274061,0.667615,-189.147301,100.0,-4.093168,2.535382
4,-2.0,100.0,-181.670449,0.667615,1.000000,-0.522439,0.121601,0.667615,-181.670449,100.0,-4.093168,2.538177
5,0.0,100.0,-175.971084,0.667615,0.980818,-0.634460,-0.023118,0.667615,-175.971084,100.0,-4.093168,2.518996
6,2.0,100.0,-171.317159,0.667615,0.947799,-0.725933,-0.157645,0.667615,-171.317159,100.0,-4.093168,2.485976
7,4.0,100.0,-167.307488,0.667615,0.906015,-0.804743,-0.281508,0.667615,-167.307488,100.0,-4.093168,2.444192
8,6.0,100.0,-163.720601,0.667615,0.858734,-0.875243,-0.395156,0.667615,-163.720601,100.0,-4.093168,2.396912
